In [1]:
# Import our dependencies
import numpy as np
import pandas as pd
import polars as pl
import re

In [2]:
#  Import and read the input csv
wbHNP_df = pd.read_csv('../../Raw_Data/World_Bank-Extract_From_Health_Nutrition_and_Population_Statistics-1990_2022.csv')
wbHNP_df.head(10)

,Series Name,Series Code,Country,Country Code,1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],...,2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022]
0,Capital health expenditure (% of GDP),SH.XPD.KHEX.GD.ZS,Afghanistan,AFG,..,..,..,..,..,..,...,0.18895136,0.164755,0.21357547,..,0.28592768,0.36722939,0.31406657,0.41934217,0.41568576,..
1,Capital health expenditure (% of GDP),SH.XPD.KHEX.GD.ZS,Albania,ALB,..,..,..,..,..,..,...,0.14006858,0.14262114,0.13107376,0.14689514,0.14722906,0.16765136,..,..,..,..
2,Capital health expenditure (% of GDP),SH.XPD.KHEX.GD.ZS,Algeria,DZA,..,..,..,..,..,..,...,..,..,0.01103354,..,..,..,..,..,..,..
3,Capital health expenditure (% of GDP),SH.XPD.KHEX.GD.ZS,American Samoa,ASM,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
4,Capital health expenditure (% of GDP),SH.XPD.KHEX.GD.ZS,Andorra,AND,..,..,..,..,..,..,...,0.04990563,0.07309079,0.11930761,0.14139073,0.10919669,0.06971787,0.10289453,..,..,..
5,Capital health expenditure (% of GDP),SH.XPD.KHEX.GD.ZS,Angola,AGO,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
6,Capital health expenditure (% of GDP),SH.XPD.KHEX.GD.ZS,Antigua and Barbuda,ATG,..,..,..,..,..,..,...,..,..,0.04921389,0.14435445,0.00550222,0.02265624,0.02125437,0.00856461,..,..
7,Capital health expenditure (% of GDP),SH.XPD.KHEX.GD.ZS,Argentina,ARG,..,..,..,..,..,..,...,0.03883833,0.03190604,0.0230072,0.03173462,0.00970424,..,..,..,..,..
8,Capital health expenditure (% of GDP),SH.XPD.KHEX.GD.ZS,Armenia,ARM,..,..,..,..,..,..,...,0.03661397,0.07182167,0.10058226,0.14318842,0.06770249,0.02918381,0.03048215,0.05976043,..,..
9,Capital health expenditure (% of GDP),SH.XPD.KHEX.GD.ZS,Aruba,ABW,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..


In [3]:
# Drop the extra columns
wbHNP_df = wbHNP_df.drop(['Country Code','Series Code'], axis=1)
wbHNP_df.head(10)

,Series Name,Country,1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],1996 [YR1996],1997 [YR1997],...,2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022]
0,Capital health expenditure (% of GDP),Afghanistan,..,..,..,..,..,..,..,..,...,0.18895136,0.164755,0.21357547,..,0.28592768,0.36722939,0.31406657,0.41934217,0.41568576,..
1,Capital health expenditure (% of GDP),Albania,..,..,..,..,..,..,..,..,...,0.14006858,0.14262114,0.13107376,0.14689514,0.14722906,0.16765136,..,..,..,..
2,Capital health expenditure (% of GDP),Algeria,..,..,..,..,..,..,..,..,...,..,..,0.01103354,..,..,..,..,..,..,..
3,Capital health expenditure (% of GDP),American Samoa,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
4,Capital health expenditure (% of GDP),Andorra,..,..,..,..,..,..,..,..,...,0.04990563,0.07309079,0.11930761,0.14139073,0.10919669,0.06971787,0.10289453,..,..,..
5,Capital health expenditure (% of GDP),Angola,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
6,Capital health expenditure (% of GDP),Antigua and Barbuda,..,..,..,..,..,..,..,..,...,..,..,0.04921389,0.14435445,0.00550222,0.02265624,0.02125437,0.00856461,..,..
7,Capital health expenditure (% of GDP),Argentina,..,..,..,..,..,..,..,..,...,0.03883833,0.03190604,0.0230072,0.03173462,0.00970424,..,..,..,..,..
8,Capital health expenditure (% of GDP),Armenia,..,..,..,..,..,..,..,..,...,0.03661397,0.07182167,0.10058226,0.14318842,0.06770249,0.02918381,0.03048215,0.05976043,..,..
9,Capital health expenditure (% of GDP),Aruba,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..


In [4]:
pattern = r'\[.*\]'

# Remove the bracketed year info from the column names
wbHNP_df.columns=[re.sub(pattern,'',col)for col in wbHNP_df.columns]
wbHNP_df.columns=wbHNP_df.columns.str.strip()
wbHNP_df.head(5)

,Series Name,Country,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Capital health expenditure (% of GDP),Afghanistan,..,..,..,..,..,..,..,..,...,0.18895136,0.164755,0.21357547,..,0.28592768,0.36722939,0.31406657,0.41934217,0.41568576,..
1,Capital health expenditure (% of GDP),Albania,..,..,..,..,..,..,..,..,...,0.14006858,0.14262114,0.13107376,0.14689514,0.14722906,0.16765136,..,..,..,..
2,Capital health expenditure (% of GDP),Algeria,..,..,..,..,..,..,..,..,...,..,..,0.01103354,..,..,..,..,..,..,..
3,Capital health expenditure (% of GDP),American Samoa,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
4,Capital health expenditure (% of GDP),Andorra,..,..,..,..,..,..,..,..,...,0.04990563,0.07309079,0.11930761,0.14139073,0.10919669,0.06971787,0.10289453,..,..,..


In [5]:
# Open the list of country name corrections
corrections_df = pd.read_csv('../../Clean_Data/master_country_list/country_name_corrections.csv')

# Convert the corrections dataframe to a dictionary.
correction_dict = dict(zip(corrections_df['wrong'], corrections_df['correct']))

In [6]:
# Apply the correction dictionary to fix the known errors
wbHNP_df['Country'] = wbHNP_df['Country'].replace(correction_dict)

In [7]:
wbHNP_df.head(5)

,Series Name,Country,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Capital health expenditure (% of GDP),Afghanistan,..,..,..,..,..,..,..,..,...,0.18895136,0.164755,0.21357547,..,0.28592768,0.36722939,0.31406657,0.41934217,0.41568576,..
1,Capital health expenditure (% of GDP),Albania,..,..,..,..,..,..,..,..,...,0.14006858,0.14262114,0.13107376,0.14689514,0.14722906,0.16765136,..,..,..,..
2,Capital health expenditure (% of GDP),Algeria,..,..,..,..,..,..,..,..,...,..,..,0.01103354,..,..,..,..,..,..,..
3,Capital health expenditure (% of GDP),American Samoa,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
4,Capital health expenditure (% of GDP),Andorra,..,..,..,..,..,..,..,..,...,0.04990563,0.07309079,0.11930761,0.14139073,0.10919669,0.06971787,0.10289453,..,..,..


In [8]:
# Open the master list of countries
countries_df = pd.read_csv('../../Clean_Data/master_country_list/country_profile_urls.csv')
countries_df = countries_df.drop(['profile_url'], axis=1)

In [9]:
master_countries = countries_df['country'].tolist()
master_countries

['Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Congo',
 'Democratic Republic of the Congo',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czechia',
 "Côte d'Ivoire",
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Grenada',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Guyana',
 'Haiti',
 'Honduras',
 'Hong Kong (China)',
 

In [10]:
# Create a list of wbdi countries
ps_countries = wbHNP_df['Country'].tolist()
ps_countries

['Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'British Virgin Islands',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Channel Islands',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Democratic Republic of the Congo',
 'Congo',
 'Costa Rica',
 "Côte d'Ivoire",
 'Croatia',
 'Cuba',
 'Curacao',
 'Cyprus',
 'Czechia',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Faroe Islands',
 'Fiji',
 'Finland',
 'France',
 'French Polynesia',
 '

In [11]:
# clean the FS data based on the SS country list
no_match = []

for country in ps_countries:
    if country in master_countries:
        continue
    else:
        no_match.append(country)

In [12]:
# Likely need to do manual country name clean-up for accented characters and abbreviations.
no_match

['Afghanistan',
 'American Samoa',
 'Bosnia and Herzegovina',
 'Cayman Islands',
 'Channel Islands',
 'Comoros',
 'Curacao',
 'Eritrea',
 'Faroe Islands',
 'French Polynesia',
 'Gibraltar',
 'Greenland',
 'Guam',
 'Iraq',
 "Korea, Dem. People's Rep.",
 'Kosovo',
 'Macao SAR, China',
 'Mongolia',
 'Montenegro',
 'Nauru',
 'New Caledonia',
 'Northern Mariana Islands',
 'Puerto Rico',
 'Sint Maarten (Dutch part)',
 'Somalia',
 'South Sudan',
 'St. Martin (French part)',
 'Timor-Leste',
 'Tonga',
 'Turks and Caicos Islands',
 'Tuvalu',
 'United Arab Emirates',
 'Virgin Islands (U.S.)',
 'West Bank and Gaza',
 'Afghanistan',
 'American Samoa',
 'Bosnia and Herzegovina',
 'Cayman Islands',
 'Channel Islands',
 'Comoros',
 'Curacao',
 'Eritrea',
 'Faroe Islands',
 'French Polynesia',
 'Gibraltar',
 'Greenland',
 'Guam',
 'Iraq',
 "Korea, Dem. People's Rep.",
 'Kosovo',
 'Macao SAR, China',
 'Mongolia',
 'Montenegro',
 'Nauru',
 'New Caledonia',
 'Northern Mariana Islands',
 'Puerto Rico',
 'S

In [13]:
wbHNP_clean_df = wbHNP_df.drop(wbHNP_df[wbHNP_df['Country'].isin(no_match)].index.tolist())
wbHNP_clean_df.head(5)

,Series Name,Country,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
1,Capital health expenditure (% of GDP),Albania,..,..,..,..,..,..,..,..,...,0.14006858,0.14262114,0.13107376,0.14689514,0.14722906,0.16765136,..,..,..,..
2,Capital health expenditure (% of GDP),Algeria,..,..,..,..,..,..,..,..,...,..,..,0.01103354,..,..,..,..,..,..,..
4,Capital health expenditure (% of GDP),Andorra,..,..,..,..,..,..,..,..,...,0.04990563,0.07309079,0.11930761,0.14139073,0.10919669,0.06971787,0.10289453,..,..,..
5,Capital health expenditure (% of GDP),Angola,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
6,Capital health expenditure (% of GDP),Antigua and Barbuda,..,..,..,..,..,..,..,..,...,..,..,0.04921389,0.14435445,0.00550222,0.02265624,0.02125437,0.00856461,..,..


In [14]:
# Open the list of series name update list
series_updates_df = pd.read_csv('../../Clean_Data/master_country_list/series_name_updates.csv')

# Convert the series_updates dataframe to a dictionary.
series_updates_dict = dict(zip(series_updates_df['current'], series_updates_df['updated']))

In [15]:
# Apply the series_updates dictionary to reduce verbosity
wbHNP_clean_df['Series Name'] = wbHNP_clean_df['Series Name'].replace(series_updates_dict)

In [16]:
# Get a list of the columns to convert to numeric
cols = wbHNP_clean_df.columns.drop(['Country', 'Series Name'])

wbHNP_clean_df[cols] = wbHNP_clean_df[cols].apply(pd.to_numeric, errors='coerce')

In [17]:
wbHNP_clean_df.head(5)

,Series Name,Country,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
1,Capital HE (% GDP),Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.140069,0.142621,0.131074,0.146895,0.147229,0.167651,NaN,NaN,NaN,NaN
2,Capital HE (% GDP),Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.011034,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Capital HE (% GDP),Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.049906,0.073091,0.119308,0.141391,0.109197,0.069718,0.102895,NaN,NaN,NaN
5,Capital HE (% GDP),Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Capital HE (% GDP),Antigua and Barbuda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.049214,0.144354,0.005502,0.022656,0.021254,0.008565,NaN,NaN


### Manage missing values

In [18]:
# Create functions to easily repeat the processing, if multiple DataFrames.

def column_names(df):
    # Combine "Country" and "Series Name" as new column to rename
    new_column = (pl.col("Country") + "_" + pl.col("Series Name")).alias("Country_Series")

    # Add new column and place it as the first column  
    df = df.with_columns([new_column]).select([new_column] + df.columns)

    # Drop "Country" and "Series Name" columns
    df = df.drop(['Country', 'Series Name'])
    return df

def df_datatypes(df):
    # Jump through a hoop to set the correct datatype for the value columns after transposition.
    # Convert DataFrame to pandas
    df_pd = df.to_pandas()

    # Set the first row as column headers
    df_pd.columns = df_pd.iloc[0]
    df_pd = df_pd[1:]

    # Convert columns to numeric, coerce non-numeric values to NaN
    for col in df_pd.columns[0:]:
        df_pd[col] = pd.to_numeric(df_pd[col], errors='coerce')

    # Convert back to Polars DataFrame
    df = pl.from_pandas(df_pd)
    return df

def missing_values(df):
    # Fill the missing values
    df = df.interpolate()
    df = df.fill_null(strategy='backward')
    df = df.fill_null(strategy='forward')
    df = df.fill_null(-2)  # NOTE: Custom value inserted for Capital HE, where there is no country data.
    return df

In [19]:
# Create a Polars dataframe
wbHNP_pl_df = pl.DataFrame(wbHNP_clean_df)
wbHNP_pl_df.head(10)

Series Name,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Capital HE (% …","""Albania""",null,null,null,null,null,null,null,null,null,null,0.005188,0.006922,0.007207,0.011548,0.0680534,0.016539,0.060499,0.029,0.326,0.370912,0.441739,0.362211,0.203855,0.140069,0.142621,0.131074,0.146895,0.147229,0.167651,null,null,null,null
"""Capital HE (% …","""Algeria""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.011034,null,null,null,null,null,null,null
"""Capital HE (% …","""Andorra""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.175674,0.1588,0.134496,0.069039,0.05238,0.049906,0.073091,0.119308,0.141391,0.109197,0.069718,0.102895,null,null,null
"""Capital HE (% …","""Angola""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Capital HE (% …","""Antigua and Ba…",null,null,null,null,null,null,null,null,null,null,0.058309,0.067401,0.054559,0.070714,0.037977,0.066203,0.025284,0.068442,0.0660577,0.071542,0.015084,0.013081,0.032773,null,null,0.049214,0.144354,0.005502,0.022656,0.021254,0.008565,null,null
"""Capital HE (% …","""Argentina""",null,null,null,null,null,null,null,null,null,null,null,0.002826,0.002897,0.014658,0.029007,0.019758,0.013384,0.017357,0.021635,0.037085,0.025108,0.029833,0.024354,0.038838,0.031906,0.0230072,0.031735,0.009704,null,null,null,null,null
"""Capital HE (% …","""Armenia""",null,null,null,null,null,null,null,null,null,null,0.267032,0.175614,0.161618,0.198013,0.355251,0.4875426,0.684891,0.79326,0.393248,0.509764,0.176753,0.147885,0.071324,0.036614,0.071822,0.100582,0.143188,0.067702,0.029184,0.030482,0.05976,null,null
"""Capital HE (% …","""Aruba""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Capital HE (% …","""Australia""",null,null,null,null,null,null,null,null,null,null,0.465832,0.47337,0.495505,0.417987,null,null,null,null,null,null,null,null,null,0.570145,0.589368,0.6155858,0.720019,0.5034349,0.536787,0.562635,null,null,null


In [20]:
# Process the DataFrame
wbHNP_pl_df = column_names(wbHNP_pl_df)

# Transpose the DataFrame
wbHNP_pl_df = wbHNP_pl_df.transpose(include_header=True)

# Correct the DataFrame datatypes
wbHNP_pl_df = df_datatypes(wbHNP_pl_df)

# Filling missing values
wbHNP_pl_df = missing_values(wbHNP_pl_df)

# The end result
wbHNP_pl_df.head(10)

Country_Series,Albania_Capital HE (% GDP),Algeria_Capital HE (% GDP),Andorra_Capital HE (% GDP),Angola_Capital HE (% GDP),Antigua and Barbuda_Capital HE (% GDP),Argentina_Capital HE (% GDP),Armenia_Capital HE (% GDP),Aruba_Capital HE (% GDP),Australia_Capital HE (% GDP),Austria_Capital HE (% GDP),Azerbaijan_Capital HE (% GDP),Bahamas_Capital HE (% GDP),Bahrain_Capital HE (% GDP),Bangladesh_Capital HE (% GDP),Barbados_Capital HE (% GDP),Belarus_Capital HE (% GDP),Belgium_Capital HE (% GDP),Belize_Capital HE (% GDP),Benin_Capital HE (% GDP),Bermuda_Capital HE (% GDP),Bhutan_Capital HE (% GDP),Bolivia_Capital HE (% GDP),Botswana_Capital HE (% GDP),Brazil_Capital HE (% GDP),British Virgin Islands_Capital HE (% GDP),Brunei_Capital HE (% GDP),Bulgaria_Capital HE (% GDP),Burkina Faso_Capital HE (% GDP),Burundi_Capital HE (% GDP),Cabo Verde_Capital HE (% GDP),Cambodia_Capital HE (% GDP),Cameroon_Capital HE (% GDP),Canada_Capital HE (% GDP),Central African Republic_Capital HE (% GDP),Chad_Capital HE (% GDP),Chile_Capital HE (% GDP),…,Seychelles_Alcohol Consumption per capita (liters),Sierra Leone_Alcohol Consumption per capita (liters),Singapore_Alcohol Consumption per capita (liters),Slovakia_Alcohol Consumption per capita (liters),Slovenia_Alcohol Consumption per capita (liters),Solomon Islands_Alcohol Consumption per capita (liters),South Africa_Alcohol Consumption per capita (liters),Spain_Alcohol Consumption per capita (liters),Sri Lanka_Alcohol Consumption per capita (liters),Saint Kitts and Nevis_Alcohol Consumption per capita (liters),Saint Lucia_Alcohol Consumption per capita (liters),Saint Vincent and the Grenadines_Alcohol Consumption per capita (liters),Sudan_Alcohol Consumption per capita (liters),Suriname_Alcohol Consumption per capita (liters),Sweden_Alcohol Consumption per capita (liters),Switzerland_Alcohol Consumption per capita (liters),Syria_Alcohol Consumption per capita (liters),Tajikistan_Alcohol Consumption per capita (liters),Tanzania_Alcohol Consumption per capita (liters),Thailand_Alcohol Consumption per capita (liters),Togo_Alcohol Consumption per capita (liters),Trinidad and Tobago_Alcohol Consumption per capita (liters),Tunisia_Alcohol Consumption per capita (liters),Türkiye_Alcohol Consumption per capita (liters),Turkmenistan_Alcohol Consumption per capita (liters),Uganda_Alcohol Consumption per capita (liters),Ukraine_Alcohol Consumption per capita (liters),United Kingdom_Alcohol Consumption per capita (liters),United States_Alcohol Consumption per capita (liters),Uruguay_Alcohol Consumption per capita (liters),Uzbekistan_Alcohol Consumption per capita (liters),Vanuatu_Alcohol Consumption per capita (liters),Venezuela_Alcohol Consumption per capita (liters),Vietnam_Alcohol Consumption per capita (liters),Yemen_Alcohol Consumption per capita (liters),Zambia_Alcohol Consumption per capita (liters),Zimbabwe_Alcohol Consumption per capita (liters)
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1990,0.005188,0.011034,0.175674,-2.0,0.058309,0.002826,0.267032,-2.0,0.465832,0.619006,0.826621,0.081105,-2.0,0.160582,0.108155,0.505285,-2.0,0.20873,0.037782,-2.0,1.62792,0.476674,0.318698,0.640317,-2.0,0.200891,0.192001,0.287929,0.025783,0.3792021,1.868197,0.078122,0.358526,0.005856,0.767958,0.102075,…,7.62,5.8,2.49,12.47,12.6,1.2,10.02,12.43,2.22,9.51,11.8,5.01,0.52,5.29,8.53,12.49,1.15,2.97,8.63,6.49,1.79,5.27,1.36,2.41,4.82,13.61,12.46,13.73,9.19,8.09,5.68,1.17,9.15,2.21,0.79,4.05,2.46
1991,0.005188,0.011034,0.175674,-2.0,0.058309,0.002826,0.267032,-2.0,0.465832,0.619006,0.826621,0.081105,-2.0,0.160582,0.108155,0.505285,-2.0,0.20873,0.037782,-2.0,1.62792,0.476674,0.318698,0.640317,-2.0,0.200891,0.192001,0.287929,0.025783,0.3792021,1.868197,0.078122,0.358526,0.005856,0

In [21]:
# Convert the Polars DataFrame to Pandas

df = wbHNP_pl_df.to_pandas()

# Rename the Country_Series column to "Year"
df.rename(columns={"Country_Series": "Year"}, inplace=True)

# Melt the DataFrame from Wide to Long format
df = df.melt(id_vars='Year', var_name='Country_Series', value_name='Value')

# Separate the Country from the Series Name
df[['Country', 'Series']] = df['Country_Series'].str.rsplit(pat='_', n=1, expand=True)

# Create a "country_year" column
df['Country_Year'] = df['Country'] + '_' + df['Year'].astype(str)

# Pivot the DataFrame to create tidy data
df_final = df.pivot(index='Country_Year', columns='Series', values='Value')
df_final.reset_index(inplace=True)

df_final

Series,Country_Year,Alcohol Consumption per capita (liters),Capital HE (% GDP),LEx years,Total Population
0,Albania_1990,6.57,0.005188,73.144,3286542.0
1,Albania_1991,6.57,0.005188,73.378,3266790.0
2,Albania_1992,6.57,0.005188,73.715,3247039.0
3,Albania_1993,6.57,0.005188,73.939,3227287.0
4,Albania_1994,6.57,0.005188,74.131,3207536.0
...,...,...,...,...,...
6034,Zimbabwe_2018,4.67,0.172716,61.414,15052184.0
6035,Zimbabwe_2019,4.67,1.526442,61.292,15354608.0
6036,Zimbabwe_2020,4.67,0.118737,61.124,15669666.0
6037,Zimbabwe_2021,4.67,0.118737,59.253,15993524.0


In [22]:
# There is data overlap with WHO.  Drop the redundant data
df_final = df_final.drop(['Alcohol Consumption per capita (liters)','LEx years','Total Population'], axis=1)

In [23]:
df_final

Series,Country_Year,Capital HE (% GDP)
0,Albania_1990,0.005188
1,Albania_1991,0.005188
2,Albania_1992,0.005188
3,Albania_1993,0.005188
4,Albania_1994,0.005188
...,...,...
6034,Zimbabwe_2018,0.172716
6035,Zimbabwe_2019,1.526442
6036,Zimbabwe_2020,0.118737
6037,Zimbabwe_2021,0.118737


In [24]:
#wbHNP_pl_df.write_csv('../../Clean_Data/Clean_CSV_Files/world_bank_Health_Nutrition_and_Population_Statistics.csv', has_header=True, quote_style='necessary')

In [25]:
df_final.to_csv('../../Clean_Data/Clean_CSV_Files/world_bank_Health_Nutrition_and_Population_Statistics.csv', index=False)

In [26]:
#fsle_clean_transform_df = fsle_clean_df.set_index('Entity').T
#fsle_clean_transform_df

In [27]:
#fsle_clean_transform_df.to_csv('food_supply_life_expectancy_transform.csv')